In [ ]:
import os
import datetime
from shutil import copyfile
import sys
import math 
import traci
import numpy as np
import random
import timeit

from generator import TrafficGenerator
from memory import Memory     ## Prority Experience Memory 
from visualization import Visualization
from utils import import_train_configuration,set_sumo, set_train_path,get_model_path

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch 
import torch.nn as nn
import torch.autograd as autograd 
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [ ]:
# phase codes based on SUMO environment.net.xml 
PHASE_NS_GREEN = 0  # action 0 for Variable Order
PHASE_NS_YELLOW = 1
PHASE_NSL_GREEN = 2  # action 1 for Variable Order
PHASE_NSL_YELLOW = 3
PHASE_EW_GREEN = 4  # action 2 for Variable Order
PHASE_EW_YELLOW = 5
PHASE_EWL_GREEN = 6  # action 3 for Variable Order
PHASE_EWL_YELLOW = 7

In [ ]:
def _get_state():
    """
    Retrieve the state of the intersection from sumo, in the form of cell occupancy
    """
    state = np.zeros((3, 208, 206))   ## kind of like an RGB image
    #Old version
    #lane = ["N2TL_0","N2TL_1","N2TL_2","E2TL_0","E2TL_1","E2TL_2","E2TL_3","S2TL_0","S2TL_1","S2TL_2","W2TL_0","W2TL_1","W2TL_2","W2TL_3"] #Clockwise
    #Now is a dictionary
    #lane     = {item: index for index, item in enumerate(["N2TL_0","N2TL_1","N2TL_2","E2TL_0","E2TL_1","E2TL_2","E2TL_3","S2TL_0","S2TL_1","S2TL_2","W2TL_0","W2TL_1","W2TL_2","W2TL_3"])}
    # N, E, S, W
    #           N
    #   W               E
    #           S
    
    car_list = traci.vehicle.getIDList()
    lane_group = 0

    for car_id in car_list:
        lane_pos = traci.vehicle.getLanePosition(car_id)
        car_speed = traci.vehicle.getSpeed(car_id)
        lane_id = traci.vehicle.getLaneID(car_id)

        # inversion of lane pos, so if the car is close to the traffic light -> lane_pos = 0 --- 750 = max len of a road
        #lane_pos = 750 - lane_pos
        
        # distance in meters from the traffic light -> mapping into cells
        #lane_cell = int(lane_pos/7.5)

        #for i in range(len(lane)):
        #    if lane_id == lane[i]:
        #        lane_group = i
        #lane_group = lane[lane_id]
        if 'N' in lane_id:            
            x = 100 + int(lane_id[-1])
            y = int(lane_pos//7.5)
            
        if 'E' in lane_id:
            x = 205 - int(pos//7.5)
            y = 99 + int(lane_id[-1])

        if 'S' in lane_id:
            x = 99 + 3 + int(lane_id[-1])
            y = 207 - int(pos//7.5)

        if 'W' in lane_id:
            x = int(lane_pos//7.5)
            y = 99 + 4 + int(lane_id[-1])

        state[0][y][x] = 1 #presence / volume
        state[1][y][x] = car_speed #velocity
        state[2][y][x] = traci.vehicle.getAccumulatedWaitingTime(car_id) #waitingTime

        #state[0][lane_group][lane_cell] = 1 
        #state[1][lane_group][lane_cell] = car_speed 
        #state[2][lane_group][lane_cell] = traci.vehicle.getAccumulatedWaitingTime(car_id) 
        
    return state.tolist()